# Implementación de ResNet

En este ejercicio práctico vamos a implementar una red neuronal con conexiones residuales y vamos a probar como dichas conexiones evitan que la performance se degrade al aumentar las capas de la red neuronal.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchsummary

Para este caso, vamos a entrenar a las redes neuronales para que aproximen la función seno, por lo que vamos a generar datos utilizando dicha función. El dataset esta generado en una clase propia, que hereda de la clase base Dataset que provee PyTorch. Para crear un dataset de esta forma, es necesario implementar los metodos `__len__` y `__getitem__`. Ver [ejemplo](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

In [ ]:
class SinDataset(torch.utils.data.Dataset):
    def __init__(self, mean, stddev):
        self.mean = mean
        self.stddev = stddev

        self.data = np.random.normal(loc=self.mean, scale=self.stddev, size=(640, 10)).astype(np.float32)
        self.target = np.sin(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, id):
        if torch.is_tensor(id):
            id = id.tolist()

        data = self.data[id]
        target = self.target[id]

        return (data, target)

In [ ]:
train_set = SinDataset(np.pi, 1.5)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

## ResNet vs Red "plana"

Con el objetivo de probar el funcionamiento de las conexiones residuales, generamos modelos "planos", es decir, sin dichas conexiones y los entrenamos aumentando cada vez mas la cantidad de capas densas en los mismos. Luego, repetimos el proceso para modelos de iguales cantidades de capas pero agregando las conexiones residuales. Finalmente comparamos la evolución del error al final del entrenamiento, a medida que se van incrementando la cantidad de capas, para ambos casos.


In [ ]:
# Defino la cantidad de capas que quiero que tenga la red mas grande
layers_amount = 25

### Entrenamiento red "plana"

In [ ]:
class PlainModel(torch.nn.Module):
    def __init__(self, layers, input_size):
        super().__init__()
        
        self.first_layer = torch.nn.Linear(in_features=input_size, out_features=10)
        self.hidden_layers = torch.nn.ModuleList()
        for i in range(layers-1):
            self.hidden_layers.append(torch.nn.Linear(in_features=10, out_features=10))

    def forward(self, x):

        x = torch.relu(self.first_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        return x

In [ ]:
plain_model = PlainModel(layers=5, input_size=10)
torchsummary.summary(plain_model, input_size=(1, 10))

In [ ]:
# Funcion de entrenamiento

def train(model, data_loader, epochs):

    optimizer = torch.optim.Adam(model.parameters())
    loss = torch.nn.MSELoss()

    for epoch in range(epochs):

        epoch_loss = 0.0
        for data, target in data_loader:

            optimizer.zero_grad()
            output = model(data)
            loss_value = loss(output, target)
            epoch_loss += loss_value.item()
            loss_value.backward()
            optimizer.step()

    last_epoch_loss = epoch_loss / len(data_loader)
    print("Error de ultima epoca es {}".format(last_epoch_loss))
    return last_epoch_loss

In [ ]:
print("Entrenamiento redes neuronales planas")

plain_loss_history = []

for layers in range(1, layers_amount):

    plain_model = PlainModel(layers=layers, input_size=10)
    plain_loss = train(plain_model, train_loader, 20*layers)
    plain_loss_history.append(plain_loss)
    del plain_model

### Entrenamiento de ResNet

In [ ]:
class ResNetBlock(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        
        self.layer1 = torch.nn.Linear(in_features, out_features)
        self.layer2 = torch.nn.Linear(in_features, out_features)

    def forward(self, x):

        shortcut = x
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        x = shortcut + x
        x = torch.relu(x)
        return x

class ResNetModel(torch.nn.Module):
    def __init__(self, layers, input_size, resnet_block):
        super().__init__()
        
        self.first_layer = torch.nn.Linear(in_features=input_size, out_features=10)
        self.hidden_layers = torch.nn.ModuleList()
        for i in range(int((layers-1)/2)):
            self.hidden_layers.append(resnet_block(in_features=10, out_features=10))

    def forward(self, x):

        x = torch.relu(self.first_layer(x))
        for layer in self.hidden_layers:
            x = layer(x)
        return x

In [ ]:
resnet_model = ResNetModel(layers=5, input_size=10, resnet_block=ResNetBlock)
torchsummary.summary(resnet_model, input_size=(1, 10))
print(resnet_model.modules)

In [ ]:
print("Entrenamiento redes neuronales residuales")

resnet_loss_history = []

for layers in range(1, layers_amount):

    resnet_model = ResNetModel(layers=layers, input_size=10, resnet_block=ResNetBlock)
    resnet_loss = train(resnet_model, train_loader, 20*layers)
    resnet_loss_history.append(resnet_loss)
    del resnet_model

## Grafica con los resultados

En la siguiente grafica se puede observar como evoluciona el error de entrenamiento a medida que se incrementa la cantidad de capas dentro de la red.

In [ ]:
plt.plot(range(1, layers_amount), plain_loss_history, label="PlainNet")
plt.plot(range(1, layers_amount), resnet_loss_history, label="ResNet")
plt.title("función seno con redes neuronales")
plt.xlabel("# de capas")
plt.ylabel("valor de loss function final")
plt.legend()
plt.show()